Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Problem1_a : L2 regularization for logistic model

In [6]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Running SGD logistic regression with regularization

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 21.300457
Minibatch accuracy: 7.8%
Validation accuracy: 15.3%
Minibatch loss at step 500: 2.847647
Minibatch accuracy: 75.8%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 1.612741
Minibatch accuracy: 75.8%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 1.404235
Minibatch accuracy: 77.3%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.800256
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 1.050620
Minibatch accuracy: 77.3%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 1.060557
Minibatch accuracy: 78.1%
Validation accuracy: 81.4%
Test accuracy: 88.3%


## Problem 1_b : L2 regularization for neural network model

In [8]:
hidden_size = 1024
batch_size = 128
beta_1 = 0.001
beta_2 = 0.001

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    W_1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_size]))
    b_1 = tf.Variable(tf.zeros(hidden_size))
    W_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
    b_2 = tf.Variable(tf.zeros(num_labels))
    
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset,W_1)+b_1),W_2) + b_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta_1*tf.nn.l2_loss(W_1)
    +beta_2*tf.nn.l2_loss(W_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,W_1)+b_1),W_2) + b_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,W_1)+b_1),W_2) + b_2) 

Running neural network model with regularization

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 631.567627
Minibatch accuracy: 7.0%
Validation accuracy: 34.2%
Minibatch loss at step 500: 192.902008
Minibatch accuracy: 82.0%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 115.870720
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 71.829506
Minibatch accuracy: 76.6%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 41.149563
Minibatch accuracy: 90.6%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.637873
Minibatch accuracy: 80.5%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.549708
Minibatch accuracy: 82.0%
Validation accuracy: 86.8%
Test accuracy: 92.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

## Neural network model with 5 batches

In [15]:
num_steps = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 605.754456
Minibatch accuracy: 13.3%
Validation accuracy: 24.4%
Test accuracy: 63.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

## Problem 3_a : Dropout on hidden layer

In [18]:
hidden_size = 1024
batch_size = 128
beta_1 = 0.001
beta_2 = 0.001

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    W_1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_size]))
    b_1 = tf.Variable(tf.zeros(hidden_size))
    W_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
    b_2 = tf.Variable(tf.zeros(num_labels))
    
    dropout_prob = tf.placeholder(tf.float32)
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,W_1) + b_1)
    hidden_drop = tf.nn.dropout(hidden,dropout_prob)
    logits = tf.matmul(hidden_drop,W_2) + b_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta_1*tf.nn.l2_loss(W_1)
    +beta_2*tf.nn.l2_loss(W_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,W_1)+b_1),W_2) + b_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,W_1)+b_1),W_2) + b_2)

In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_prob: 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 673.459229
Minibatch accuracy: 14.8%
Validation accuracy: 26.1%
Minibatch loss at step 500: 203.777100
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 118.698593
Minibatch accuracy: 78.1%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 72.562149
Minibatch accuracy: 73.4%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 41.114754
Minibatch accuracy: 88.3%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 26.316059
Minibatch accuracy: 80.5%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.578151
Minibatch accuracy: 79.7%
Validation accuracy: 86.6%
Test accuracy: 92.3%


## Problem 3_b : Dropout with extreme overfit

In [25]:
num_steps = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_prob: 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 698.345032
Minibatch accuracy: 8.6%
Validation accuracy: 22.3%
Minibatch loss at step 1: 1928.330078
Minibatch accuracy: 16.4%
Validation accuracy: 34.2%
Minibatch loss at step 2: 1667.745361
Minibatch accuracy: 32.0%
Validation accuracy: 39.5%
Minibatch loss at step 3: 1810.268799
Minibatch accuracy: 36.7%
Validation accuracy: 49.3%
Minibatch loss at step 4: 871.807983
Minibatch accuracy: 46.1%
Validation accuracy: 53.1%
Test accuracy: 58.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## Problem 4_a : Multi-layer model

In [81]:
batch_size = 256
num_labels = 10
hidden_size_1 = 1024
hidden_size_2 = 750
hidden_size_3 = 256

beta_1 = 1e-5
beta_2 = 1e-5
beta_3 = 1e-5
beta_4 = 1e-5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size_1], stddev=tf.sqrt(2./(image_size * image_size))))
  b_1 = tf.Variable(tf.zeros([hidden_size_1]))

  W_2 = tf.Variable(tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=tf.sqrt(2./(hidden_size_1))))
  b_2 = tf.Variable(tf.zeros([hidden_size_2]))
  
  W_3 = tf.Variable(tf.truncated_normal([hidden_size_2, hidden_size_3], stddev=tf.sqrt(2./(hidden_size_2))))
  b_3 = tf.Variable(tf.zeros([hidden_size_3]))

  W_4 = tf.Variable(tf.truncated_normal([hidden_size_3, num_labels], stddev=tf.sqrt(2./(hidden_size_3))))
  b_4 = tf.Variable(tf.zeros([num_labels]))
    
  dropout_prob = tf.placeholder(tf.float32, shape=3)

  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, W_1) + b_1)
  hidden_2 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_1, dropout_prob[0]), W_2) + b_2)
  hidden_3 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_2, dropout_prob[1]), W_3) + b_3)
  logits = tf.matmul(tf.nn.dropout(hidden_3, dropout_prob[2]), W_4) + b_4
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
    + beta_1 * (tf.nn.l2_loss(W_1) + beta_2*tf.nn.l2_loss(W_2) + beta_3*tf.nn.l2_loss(W_3)) + beta_4 * tf.nn.l2_loss(W_4)
    
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(
              tf.nn.relu(tf.matmul(
                  tf.nn.relu(tf.matmul(tf_valid_dataset, W_1) + b_1), W_2) + b_2), W_3) + b_3), W_4) + b_4))
  test_prediction = tf.nn.softmax(
      tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(
              tf.nn.relu(tf.matmul(
                  tf.nn.relu(tf.matmul(tf_test_dataset, W_1) + b_1), W_2) + b_2), W_3) + b_3), W_4) + b_4))

In [82]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_prob: [1.0,1.0,1.0]}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.402032
Minibatch accuracy: 9.0%
Validation accuracy: 18.6%
Minibatch loss at step 500: 0.398128
Minibatch accuracy: 86.7%
Validation accuracy: 87.4%
Minibatch loss at step 1000: 0.276257
Minibatch accuracy: 92.2%
Validation accuracy: 88.8%
Minibatch loss at step 1500: 0.330157
Minibatch accuracy: 90.2%
Validation accuracy: 89.3%
Minibatch loss at step 2000: 0.322637
Minibatch accuracy: 90.2%
Validation accuracy: 89.3%
Minibatch loss at step 2500: 0.151529
Minibatch accuracy: 96.5%
Validation accuracy: 90.3%
Minibatch loss at step 3000: 0.296769
Minibatch accuracy: 90.2%
Validation accuracy: 90.2%
Minibatch loss at step 3500: 0.199418
Minibatch accuracy: 94.1%
Validation accuracy: 89.9%
Minibatch loss at step 4000: 0.215513
Minibatch accuracy: 94.1%
Validation accuracy: 89.9%
Minibatch loss at step 4500: 0.184380
Minibatch accuracy: 94.5%
Validation accuracy: 90.8%
Minibatch loss at step 5000: 0.156793
Minibatch accuracy: 95.3%
Validation accuracy

## Problem 4_b: Learning rate decay

In [85]:
hidden_size = 1024
batch_size = 128
beta_1 = 0.001
beta_2 = 0.001

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    W_1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_size]))
    b_1 = tf.Variable(tf.zeros(hidden_size))
    W_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
    b_2 = tf.Variable(tf.zeros(num_labels))
    
    dropout_prob = tf.placeholder(tf.float32)
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,W_1) + b_1)
    hidden_drop = tf.nn.dropout(hidden,dropout_prob)
    logits = tf.matmul(hidden_drop,W_2) + b_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta_1*tf.nn.l2_loss(W_1)
    +beta_2*tf.nn.l2_loss(W_2)
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, global_step,1000,0.9)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,W_1)+b_1),W_2) + b_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,W_1)+b_1),W_2) + b_2)

In [86]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_prob: 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 656.913330
Minibatch accuracy: 14.8%
Validation accuracy: 24.9%
Minibatch loss at step 500: 201.062958
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 122.372787
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 80.171768
Minibatch accuracy: 75.8%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 50.153557
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 34.655357
Minibatch accuracy: 79.7%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 23.557348
Minibatch accuracy: 78.9%
Validation accuracy: 86.2%
Test accuracy: 92.4%
